# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [4]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

In [5]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

df_densidad_alta = pd.read_csv("instancias/densidad-alta/indice.csv");
df_densidad_baja = pd.read_csv("instancias/densidad-baja/indice.csv");
df_mejor_caso_bt = pd.read_csv("instancias/mejor-caso-bt/indice.csv");
dataset_4 = pd.read_csv("instancias/dataset-4/indice.csv");
dataset_5 = pd.read_csv("instancias/dataset-5/indice.csv");

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.
- FB: Fuerza bruta
- BT: Backtracking con ambas podas.
- BT-F: Backtracking solamente con poda por factibilidad.
- BT-O: Backtracking solamente con poda por optimalidad.
- DP: Programación dinámica.

In [6]:
def correr_experimento(metodo, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../npm", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return tiempo_de_ejecucion;

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [7]:
experimentos = [];

### Experimento 1
Correr Fuerza Bruta en las primeras 30 instancias de densidad-alta y densidad-baja.

In [8]:
for n in range(0, 30):
    fila_n = df_densidad_alta.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "FB", fila_n["archivo"]]);

for n in range(0, 30):
    fila_n = df_densidad_baja.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "FB", fila_n["archivo"]]);

## Experimento 2
Correr BT para mejor y peor caso.

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [9]:
columnas = ["dataset", "n", "W", "metodo", "tiempo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    for i in range(0, T):
        tiempos.append(correr_experimento(experimento[3], experimento[4]));
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado.csv", index=False, header=True);

'Experimento: 60/60'

FileNotFoundError: [Errno 2] No such file or directory: 'resultados/resultado.csv'